In [39]:
import sys
import os
from scipy.io import wavfile
import numpy as np
data_folder = "/Volumes/CB_RESEARCH/vocal_synthesis/"
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.init import *
from lasagne.nonlinearities import *

In [40]:
import pylearn2

In [14]:
fs, data = wavfile.read(data_folder + os.path.sep + "data.wav")

In [12]:
fs # sample rate, in samples per sec

16000

In [15]:
data.shape

(174229537,)

In [22]:
np.min(data), np.max(data)

(-8936, 9124)

We want the output of the network to be a mean vector and a variance term (for the covariance matrix)

Likelihood: $P(x^{(t+1)} \ | \ x^{(1)}, \dots, x^{(t)})$

https://www.cs.cmu.edu/~epxing/Class/10701-08s/recitation/gaussian.pdf

http://www.maths.manchester.ac.uk/~mkt/MT3732%20(MVA)/Notes/MVA_Section3.pdf

Suppose our block of data is $x = \{x^{(1)}, x^{(2)}, \dots, x^{(m)}\}$, where the block is $m$ long and drawn from $N(x; \mu, \sigma)$. We would like to estimate $\mu$ and $\sigma$.

----

In [36]:
x = T.fmatrix('x')
y = T.fvector('y')

In [42]:
import mnd

In [77]:
mvn = mnd.MND(mu=np.asarray([1,2]), sigma=np.asarray([[1,0],[0,1]]))

In [52]:
tmp = T.fmatrix('tmp')
mvn.log_prob(tmp)

Elemwise{sub,no_inplace}.0

In [58]:
example_data = data[5000:5020]
example_data

array([419, 389, 336, 289, 229, 220, 249, 276, 346, 427, 548, 624, 674,
       718, 733, 719, 676, 601, 513, 463], dtype=int16)

In [65]:
def iterate_this(example_data, chunk_size=5):
    b = 0
    while True:
        if b*chunk_size >= len(example_data):
            break
        yield example_data[b*chunk_size : (b+1)*chunk_size]
        b += 1

In [66]:
for chunk in iterate_this(example_data):
    print chunk

[419 389 336 289 229]
[220 249 276 346 427]
[548 624 674 718 733]
[719 676 601 513 463]


In [67]:
prev_chunk = None
for chunk in iterate_this(example_data):
    if prev_chunk != None:
        print prev_chunk, chunk
    prev_chunk = chunk

[419 389 336 289 229] [220 249 276 346 427]
[220 249 276 346 427] [548 624 674 718 733]
[548 624 674 718 733] [719 676 601 513 463]


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  app.launch_new_instance()


http://papers.nips.cc/paper/5023-generalized-denoising-auto-encoders-as-generative-models.pdf

In [68]:
l_in = InputLayer( (None, 20) ) # given the input vector X1

In [72]:
get_out = theano.function([x], lasagne.layers.get_output(l_in, x))

In [75]:
get_out([[1,2]])

array([[ 1.,  2.]], dtype=float32)

-----

In [82]:
X_train = np.asarray(data, dtype="float32")
mean = np.mean(X_train)
std = np.std(X_train)
X_train = (X_train - mean) / std

In [88]:
X_train_actual = []
y_train_actual = []
first_time = True
prev_chunk = None
for chunk in iterate_this(X_train):
    if first_time:
        prev_chunk = chunk
        first_time = False
    else:
        X_train_actual.append(prev_chunk)
        y_train_actual.append(chunk)
        prev_chunk = chunk